# L’analyse des ventes dans une PME

*Projet pour la préparation à la journée de sélection pour le parcours de formation Développeur en intelligence artificielle*

**Darya Filatova** 



## Objectifs
* Explorer un jeu de données et en expliquer ses caractéristiques
* Lire des données depuis une table d’une base de données avec SQL
* Réaliser un premier niveau d’analyses de données avec SQL
* Rendre compte des analyses en créant des graphiques avec Python

## L’exploration d’un jeu de données

In [1]:
import plotly.express as px
import pandas as pd

### Analyse exploratoire des données
* lire de jeu de données
* visualiser les premières lignes de jeu de données
* obtenir un aperçu des colonnes et des valeurs

In [2]:
données = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSC4KusfFzvOsr8WJRgozzsCxrELW4G4PopUkiDbvrrV2lg0S19-zeryp02MC9WYSVBuzGCUtn8ucZW/pub?output=csv')
print(données.head())
print(données.info())

print('Info prix et quantité de produit')
print(données.describe())

print('Info date')
print(données['date'].value_counts().sort_index())
print('Info produit')
print(données['produit'].value_counts())
print('Info region')
print(données['region'].value_counts())

         date    produit  prix  qte region
0  2022-01-01  Produit A    10  100   Nord
1  2022-01-02  Produit B    15   50   Nord
2  2022-01-02  Produit A    10   75    Sud
3  2022-01-03  Produit C    20   30   Nord
4  2022-01-03  Produit A    10  150    Sud
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     39 non-null     object
 1   produit  39 non-null     object
 2   prix     39 non-null     int64 
 3   qte      39 non-null     int64 
 4   region   39 non-null     object
dtypes: int64(2), object(3)
memory usage: 1.7+ KB
None
Info prix et quantité de produit
            prix         qte
count  39.000000   39.000000
mean   14.743590   86.666667
std     4.127685   41.916793
min    10.000000   25.000000
25%    10.000000   55.000000
50%    15.000000   80.000000
75%    20.000000  110.000000
max    20.000000  200.000000
Info date
date
2022-01-01    1
2

**Cet ensemble de données contient 5 variables : ** 
* *'date'* - date, 
* *'produit'* - produit avec les valeurs 'Produit A', 'Produit B' et 'Produit C', 
* *'prix'* - prix du produit, 
* *'qte'* - quantité de produit vendu, 
* *'region'* - région où le produit a été vendu. 

**Les types de données peuvent poser des problèmes à l'avenir, donc 'data' doit être 'datetime', 'qte' et 'produit' doivent être 'str'. Cependant, pour l’instant, les types peuvent être conservés. Il n'y a aucune valeur manquante.**

In [3]:
# Changement de type
données['date'] = pd.to_datetime(données['date'])
données['produit'] = données['produit'].astype(str)
données.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     39 non-null     datetime64[ns]
 1   produit  39 non-null     object        
 2   prix     39 non-null     int64         
 3   qte      39 non-null     int64         
 4   region   39 non-null     object        
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 1.7+ KB


## 2. Les requêtes SQL

* le chiffre d'affaires total

SELECT SUM(prix * qte) AS le_chiffre_daffaires  FROM ventes;

* les ventes par produit,

SELECT produit, SUM(prix * qte) AS ventes_par_produit FROM ventes GROUP BY produit; 

* les ventes par région.

SELECT region, SUM(qte) AS ventes_par_region FROM ventes GROUP BY region;


### Les pandas python peuvent faire la même chose

In [4]:
#le chiffre d'affaires
données['chiffre_affaires']=données['prix']*données['qte']
print('le chiffre d\'affaires ', données['chiffre_affaires'].sum() )

le chiffre d'affaires  44825


In [5]:
#les ventes par produit
grouped_df01 =données.groupby("produit")["qte"].sum()
new_df01 = pd.DataFrame(grouped_df01).reset_index()
new_df01.columns = ["produit", "ventes"]
print(new_df01)

     produit  ventes
0  Produit A    1750
1  Produit B    1055
2  Produit C     575


In [6]:
#les ventes par region
grouped_df02 =données.groupby("region")["qte"].sum()
new_df02 = pd.DataFrame(grouped_df02).reset_index()
new_df02.columns = ["region", "ventes"]
print(new_df02)

  region  ventes
0   Nord    1605
1    Sud    1775


In [7]:
#le chiffre d'affaires par produit
grouped_df03 =données.groupby("produit")["chiffre_affaires"].sum()
new_df03 = pd.DataFrame(grouped_df03).reset_index()
new_df03.columns = ["produit", "chiffre_affaires"]
print(new_df03)

     produit  chiffre_affaires
0  Produit A             17500
1  Produit B             15825
2  Produit C             11500


## 3. Visualisation de données

In [8]:
#les ventes par region
figure = px.pie(données, values='qte', names='region', title='quantité vendue par région')
figure.write_html('ventes-par-region.html')
print('ventes-par-région.html généré avec succès !')

#les ventes par produit
figure = px.pie(données, values='qte', names='produit', title='quantité vendue par produit')
figure.write_html('ventes-par-produit.html')
print('ventes-par-produit.html généré avec succès !')

#le chiffre d'affaires par produit
figure = px.pie(données, values='chiffre_affaires', names='produit', title='chiffre d\'affaires par produit')
figure.write_html('chiffre-affaires-par-produit.html')
print('chiffre-daffaires-par-produit.html généré avec succès !')

ventes-par-région.html généré avec succès !
ventes-par-produit.html généré avec succès !
chiffre-daffaires-par-produit.html généré avec succès !


## 4. Le lien vers projet Glitch -"produit-region"- complété

projet Glitch - https://glitch.com/edit/#!/produit-region?path=README.md%3A1%3A0

Dans "Assets", vous pouvez également trouver le fichier SQL - projectMS.sql

GitHub repository  - https://github.com/delnouty/Project-for-MS

